In [ ]:

#-----------------------------Escoger modelo

#se deja inputs que el modelo va a usar para aprender menos el que quiere predecir
X = data_social.drop('actual_productivity_score', axis=1)
#se deja el que quiere predecir
y = data_social['actual_productivity_score']
X = X.select_dtypes(include=np.number)
# Dividir el dataset en conjunto de entrenamiento (70%) y prueba (30%).
# donde 70 ayuda al modelo a aprender y 30 para testear
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Aplicar los modelos de regresión
models = {
    'Linear': LinearRegression(),
    'Ridge': Ridge(alpha=1.0),
    'Lasso': Lasso(alpha=0.1),
    'Decision Tree': DecisionTreeRegressor(max_depth=8),
    'Random Forest': RandomForestRegressor(n_estimators=100, max_depth=8, random_state=42, n_jobs=-1)
    #Incluir este modelo lleva mucho tiempo de ejecución por lo que no se pudo ejecutar
    #,
    #'SVR': SVR(C=100, gamma=0.1, epsilon=0.1)
}

# Estandarizar las variables de entrada cuando sea necesario
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
results = {}

# Evaluar cada modelo
for name, model in models.items():
    print(f"Training {name}...")
    if 'SVR' in name or 'Regresión' in name:
        # optmizado para que se ejecute más rápido
        X_sampled, _, y_sampled, _ = train_test_split(X, y, test_size=0.9, random_state=42)

        # Paso 2: Escalar
        scaler = StandardScaler()
        X_sampled_scaled = scaler.fit_transform(X_sampled)

        # Paso 3: Entrenar SVR rápido
        from sklearn.svm import SVR
        svr = SVR(C=1.0, gamma='scale', epsilon=0.2)
        svr.fit(X_sampled_scaled, y_sampled)
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
    else:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
    # calcula rmse y r2 que son los valores que nos ayudan a esocger el modelo
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    results[name] = {'RMSE': rmse, 'R²': r2}

    # Graficar la comparación
    plt.figure(figsize=(8, 6))
    plt.scatter(y_test, y_pred, alpha=0.5, label='Observaciones (Valor Real vs Predicho)')
    plt.plot([y.min(), y.max()], [y.min(), y.max()], 'r--', lw=2, label='Línea Ideal (Real = Predicho)')

    plt.title(f'Valor real vs Valor predicho - {name}')
    plt.xlabel('Valor real')
    plt.ylabel('Valor predicho')
    plt.xlim(y.min(), y.max())
    plt.ylim(y.min(), y.max())
    plt.grid()
    plt.legend()
    plt.show()

# Crear tabla resumen con los resultados
results_df = pd.DataFrame(results).T
print("\nResumen de resultados:")
print(results_df)

print("\nEscogemos el modelo con un RMSE bajo y un R² alto: Ridge y Linear")


